In [14]:
import numpy as np
from tqdm import trange
from gymnasium import spaces
from gymnasium.utils.env_checker import check_env

from flygym.simulation import SingleFlySimulation, Fly
from flygym.examples.common import PreprogrammedSteps
from flygym.examples.cpg_controller import CPGNetwork
from flygym.preprogrammed import get_cpg_biases
from hybrid_turning_fly import HybridTurningFly
from pathlib import Path
import matplotlib.pyplot as plt
from flygym import Parameters, Camera, SingleFlySimulation, Fly
from flygym.examples.turning_controller import HybridTurningNMF
from flygym.simulation import Simulation
from hybrid_turning_fly import HybridTurningFly
from movodor_arena import MovOdorArena
seed = 1

# Odor source: array of shape (num_odor_sources, 3) - xyz coords of odor sources
odor_source = np.array([[24, 10, 1.5]])

# Peak intensities: array of shape (num_odor_sources, odor_dimensions)
# For each odor source, if the intensity is (x, 0) then the odor is in the 1st dimension
# (in this case attractive). If it's (0, x) then it's in the 2nd dimension (in this case
# aversive)
peak_odor_intensity = np.array([[1, 0]])

# Marker colors: array of shape (num_odor_sources, 4) - RGBA values for each marker,
# normalized to [0, 1]
marker_colors = [[255, 127, 14]]
marker_colors = np.array([[*np.array(color) / 255, 1] for color in marker_colors])

odor_dimensions = len(peak_odor_intensity[0])
arena = MovOdorArena(
    size=(300, 300),
    friction=(1, 0.005, 0.0001),
    num_sensors=4,
    move_speed=0,
    move_direction="right",
    odor_source=odor_source,
    peak_intensity=peak_odor_intensity,
    diffuse_func=lambda x: x**-2,
    marker_colors=marker_colors,
    marker_size=0.3,
)

sim_params = Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.5,
    render_window_size=(800, 608),
    enable_olfaction=True,
    enable_adhesion=True,
    draw_adhesion=False,
    render_camera="Animat/camera_right",
)
timestep = sim_params.timestep
fly_male = Fly(
    name="female",
    
    enable_adhesion=True,
    enable_olfaction=True,
    spawn_pos=(0, 0, 0.2),
)

cam = Camera(
    fly=fly_male,
    camera_id="Animat/camera_right",
    window_size=(800, 608),
    play_speed=0.5,
)

sim = SingleFlySimulation(
    fly=fly_male,
    cameras=[cam],
    arena=arena,
    timestep=timestep,
)

preprogrammed_steps = PreprogrammedSteps()
legs = preprogrammed_steps.legs


run_time=1

all_joint_angles = []
kicking_action=[]


# kick_action["joint_LHCoxa"]=np.concatenate([np.linspace(0.6, -0.2, 200),np.linspace(-0.2, -0.2, 100),np.linspace(-0.2, -0.21, 200),np.linspace(-0.2, -0.2, 300),np.linspace(-0.2,0.2, 200)]).tolist()

# kick_action["joint_LHCoxa_roll"]=np.concatenate([np.linspace(0, 0, 200), np.linspace(0, 0, 100),np.linspace(0, 0, 200),np.linspace(0, 0, 300),np.linspace(0, 0, 200)]).tolist()
# kick_action["joint_LHCoxa_yaw"]=np.concatenate([np.linspace(-0.7,-1, 200),np.linspace(-1,-1, 100),np.linspace(-1,-1, 200),np.linspace(-1, -1, 300),np.linspace(-1, -1, 200)]).tolist()
# kick_action["joint_LHFemur"]=np.concatenate([np.linspace(-2.39,-2.1, 200),np.linspace(-2.1, -2.1, 100), np.linspace(-2.1, -1.1, 200),np.linspace(-1.1,-1.1, 300),np.linspace(-1.1,-1.1, 200)]).tolist()
# kick_action["joint_LHFemur_roll"]=np.concatenate([np.linspace(0,-1, 200),np.linspace(-1, -1, 100),np.linspace(-1,-1, 200),np.linspace(-1, -1, 300),np.linspace(-1, -1, 200)]).tolist()
# kick_action["joint_LHTibia"]=np.concatenate([np.linspace(1.67,1.0, 200),np.linspace(1, 1.0, 100),np.linspace(1.0, 0.11, 200),np.linspace(0.11, 0.11, 300),np.linspace(0.11, 0.11, 200)]).tolist()
# kick_action["joint_LHTarsus1"]=np.concatenate([np.linspace(0.5,0.21 ,200),np.linspace(0.21,0.21,100),np.linspace(0.2,0.11 , 200),np.linspace(0.11, 0.11, 300),np.linspace(0.11, 0.11, 200)]).tolist()
target_num_steps = int(run_time / sim_params.timestep)
swing_periods = preprogrammed_steps.swing_period
for leg in legs:
    if leg.endswith("H"):
        kicking_action.extend(preprogrammed_steps.get_joint_angles(leg,swing_periods[leg][1]))
    else:
        kicking_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))

kick_action={'joints':kicking_action,'adhesion':np.zeros(len(legs))}
hindleg_stance_ids = np.linspace(swing_periods["RH"][1], 2 * np.pi, target_num_steps)
R_hindleg_start= preprogrammed_steps.get_joint_angles("RH", swing_periods["RH"][1])
R_hindleg_stretch=np.linspace(
    np.zeros(len(R_hindleg_start)), -R_hindleg_start, target_num_steps
)
# print("Action Shapes and Contents:")
# for key, value in kick_action.items():
#     print(f"{key}: {len(value)} elements")

run_time=1

adhesion_action = np.array([0.0 if leg.endswith("F") else 1.0 for leg in legs])
for i in range(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("H"):
            Hind_joint_angles=(preprogrammed_steps.get_joint_angles(
                leg, hindleg_stance_ids[i]
            ))
            if leg.startswith("R"):
                Hind_joint_angles += R_hindleg_stretch[i]
            joint_angles.extend(Hind_joint_angles)
        elif leg.endswith("F"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(
                leg, swing_periods[leg][1]
            ))
        else:
            joint_angles.extend(preprogrammed_steps.get_joint_angles(
                leg,swing_periods[leg][1]
            ))


    all_joint_angles.append(joint_angles.copy())

    action = {"joints": np.array(joint_angles), "adhesion": adhesion_action}
    
    sim.step(action)
    sim.render()


from IPython.display import Video

cam.save_video("./outputs/kicking_video.mp4")

Video("./outputs/kicking_video.mp4", width=800, height=608)
